In [1]:
import numpy as np
from scipy.integrate import odeint, ode
import matplotlib.pyplot as plt
from matplotlib import rcParams
import ipywidgets as widgets
%matplotlib inline

rcParams['figure.figsize'] = [15,5]

In [2]:
def plotFergulst(N0,mu,k):
    
    model = lambda N,t,mu,k: mu * N * (k - N)
    
    t = np.linspace(0, 12, 1000)
    
    N = odeint(model, N0, t, args=(mu, k))

    plt.plot(t, N)
    plt.xlabel('t', fontsize=15)
    plt.ylabel('N(t)', fontsize=15)
    plt.grid()
    plt.xlim(0,12)
    plt.ylim(2,20)
    plt.show()

In [3]:
def plotSolou(k0,s,a,mu,q,alpha):
    def model(k, t, s, a, mu, q, alpha):
        dydt = s * a * k**alpha - (mu + q) * k
        return dydt

    t = np.linspace(0, 40, 100)

    k = odeint(model, k0, t, args=(s, a, mu, q, alpha))

    plt.plot(t, k)
    plt.xlabel('t')
    plt.ylabel('k(t)')
    plt.grid()
    plt.xlim(0,40)
    plt.ylim(0,4)
    plt.show()

### модель Фергюльста динаміки росту популяцій

## $$\dot{N} = r N (k - N)$$

N – чисельність популяції,  
r – коефіцієнти народжуваності (приріст популяції за одиницю часу),  
k – ємність середовища (гранична чисельність популяції),  
N0 – початкова чисельність популяції.  

In [4]:
widgets.interact(plotFergulst,N0=(1,10),mu=(0.01,1),k=(1,30))

interactive(children=(IntSlider(value=5, description=u'N0', max=10, min=1), FloatSlider(value=0.505, description=u'mu', max=1.0, min=0.01), IntSlider(value=15, description=u'k', max=30, min=1), Output()), _dom_classes=('widget-interact',))

<function __main__.plotFergulst>

## Односекторна модель економічної динаміки Солоу 

### $$\dot{k} = sak^{\alpha} - (m + q)k$$

$\alpha$ – частка капіталу в продукції  
m – норма витрат на амортизацію  
a – рівень розвитку економіки  
s – норма накопичення  
q – темп приросту робочої сили  
k0 – початковий капітал  

In [5]:
# parameters
k0 = 0.001
s = 0.2
a = 2.5
mu = 0.1
q = 0.1
alpha = 0.3

In [6]:
widgets.interact(plotSolou,k0=(0.001,0.5),s=(0.1,1),a=(2.5,5),mu=(0.01,1),q=(0.01,1),alpha=(0,1.))

interactive(children=(FloatSlider(value=0.2505, description=u'k0', max=0.5, min=0.001), FloatSlider(value=0.55, description=u's', max=1.0, min=0.1), FloatSlider(value=3.75, description=u'a', max=5.0, min=2.5), FloatSlider(value=0.505, description=u'mu', max=1.0, min=0.01), FloatSlider(value=0.505, description=u'q', max=1.0, min=0.01), FloatSlider(value=0.5, description=u'alpha', max=1.0), Output()), _dom_classes=('widget-interact',))

<function __main__.plotSolou>

# 3. Рівняння вимушених коливань

### $$\ddot{x} + 2 \delta \dot{x} + \omega_0^2 x = f_0 \cos \omega t$$

$\omega$ – коефіцієнт згасання  
$w_0$ – власна частота коливань  
$w$ – частота коливань зовнішньої сили   
$f_0$ – амплітуда зовнішньої сили  

In [7]:
# parameters
x0 = 1
x0_prime = -0.5
delta = 0.01
omega0 = 0.7 * np.pi
f0 = 1
omega = np.sqrt(omega0**2 - 2 * delta**2)

In [8]:
def plotKol(x0, x0_prime, delta, omega0, f0):
    def model(X, t, omega, delta, omega0, f0):
        return [X[1], f0 * np.cos(omega * t) - omega0**2 * X[0] - 2 * delta * X[1]]

    t = np.linspace(0, 80, 20000)
    omega = np.sqrt(omega0**2 - 2 * delta**2)

    x = odeint(model, [x0, x0_prime], t, args=(omega, delta, omega0, f0))

    # plot results
    plt.figure(figsize=(14, 8))
    plt.plot(t, x[:, 0])
    plt.xlabel('t')
    plt.ylabel('x(t)')
    plt.grid()
    plt.xlim(0,60)
    plt.show()
    f,ax = plt.subplots(1, 1, figsize=(10,10))
    ax.plot(x[:, 0], x[:, 1], c=(1,0.7,0.5))
    ax.grid()
    ax.set_xlim(-30,30)
    ax.set_ylim(-30,30)

In [9]:
widgets.interact(plotKol, 
                 x0=(0, 10), 
                 x0_prime=(0, 10), 
                 delta=(0.005, 0.1), 
                 omega0=(0.1 * np.pi, 1 * np.pi), 
                 f0=(0, 1.))

interactive(children=(IntSlider(value=5, description=u'x0', max=10), IntSlider(value=5, description=u'x0_prime', max=10), FloatSlider(value=0.0525, description=u'delta', max=0.1, min=0.005), FloatSlider(value=1.7278759594743862, description=u'omega0', max=3.141592653589793, min=0.3141592653589793), FloatSlider(value=0.5, description=u'f0', max=1.0), Output()), _dom_classes=('widget-interact',))

<function __main__.plotKol>

# 4. Рівняння коливань у системі «хижак-жертва»

$$\displaystyle\left\{ \begin{array}{ll} \dot{x} = (\alpha_x y - \beta_x)x \space \space | \space x_0 \\ \dot{y} = (\alpha_y - \beta_y x)y \space \space | \space y_0 \end{array} \right. $$

x – кількість «хижаків»  
y – кількість «жертв»  
$a_1$ – «норма споживання» жертв  
$b_1$ – природна смертність хижаків  
$a_2$ – народжуваність жертв  
$b_2$ –«норма споживаності» жертв  
$х_0$ – початкова кількість хижаків  
$у_0$ – початкова кількість жертв  

### Стаціонарна точка: $x^* = \frac{\alpha_y}{\beta_y}; \space y^* = \frac{\beta_x}{\alpha_x}$

In [10]:
# parameters
# x0 = 2000
# y0 = 5000
alpha_x = 0.0015
beta_x = 10
alpha_y = 100
beta_y = 0.01
y0 = beta_x / alpha_x + 100000
x0 = alpha_y / beta_y + 1000

In [23]:
def plotVictim(x0, y0, alpha_x, beta_x, alpha_y, beta_y):
    def model(X, t, alpha_x, beta_x, alpha_y, beta_y):
        return [(alpha_x * X[1] - beta_x) * X[0], (alpha_y - beta_y * X[0]) * X[1]]

    # time points
    t = np.linspace(0, 1, 5000)

    # solve ODE
    x = odeint(model, [x0, y0], t, args=(alpha_x, beta_x, alpha_y, beta_y))

    # plot results
    plt.plot(t, x)
    plt.xlabel('t')
    plt.ylabel('x(t), y(t)')
    plt.xlim(0,1)
    plt.grid()
    f,ax = plt.subplots(1, 1)
    plt.plot(x[:, 0], x[:, 1])
    plt.xlabel('x(t)')
    plt.ylabel('y(t)')
    plt.grid()

In [26]:
widgets.interact(plotVictim, 
                 x0 = (0, 100), 
                 y0 = (0, 100), 
                 alpha_x = (0.01, 1), 
                 beta_x = (0, 5), 
                 alpha_y = (0, 5),
                 beta_y = (0.01, 1))

interactive(children=(IntSlider(value=50, description=u'x0'), IntSlider(value=50, description=u'y0'), FloatSlider(value=0.505, description=u'alpha_x', max=1.0, min=0.01), IntSlider(value=2, description=u'beta_x', max=5), IntSlider(value=2, description=u'alpha_y', max=5), FloatSlider(value=0.505, description=u'beta_y', max=1.0, min=0.01), Output()), _dom_classes=('widget-interact',))

<function __main__.plotVictim>